피처 벡터화 : One-hot encoding 
### Bag of Words 
- 문맥이나 순서를 무시하고 일괄적으로 단어에 대한 빈도 값을 부여해 피처 값을 추출하는 모델.
- 문맥 의미 반영 부족, 희소 행렬 문제.
### BOW에서 피처 벡터화 
- 모든 단어를 컬럼 형태로 나열하고 각 문서에서 해당 단어의 횟수나 정규화된 빈도를 값으로 부여하는 데이터 세트 모델로 변경하는 것.
### 피처 벡터화 방식 
- 카운트 기반, TF-IDF(Term Frequency - Inverse Document Frequency) 기반 벡터화
카운트 벡터화: 카운트 값이 높을수록 중요한 단어로 인식. 특성상 자주 사용되는 보편적인 단어까지 높은 값 부여
TF-IDF: 모든 문서에서 전반적으로 자주 나타나는 단어에 대해서 패널티 부여. '빈번하게', '당연히', '조직', '업무' 등
파라미터

- max_df : 너무 높은 빈도수를 가지는 단어 피처를 제외
- min_df : 너무 낮은 빈도수를 가지는 단어 피처를 제외
- max_features : 추출하는 피처의 개수를 제한하며 정수로 값을 지정
- stop_words : 'english'로 지정하면 스톱 워드로 지정된 단어는 추출에서 제외
- n_gram_range : 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정
- analyzer : 피처 추출을 수행하는 단위. 디폴트는 'word'
- token_pattern : 토큰화를 수행하는 정규 표현식 패턴을 지정
- tokenizer : 토큰화를 별도의 커스텀 함수로 이용시 적용

In [ ]:
# 피처벡터화
# 공간이 넓어지면 희소해진다.. 희소행렬.. 그래서 PCA???
# 정규표현식 -> 점프 투 파이썬 참고
# 자연어분야를 할 사람들은 정규표현식을 인텐시브하게 할 필요가 있다. 
# 자연어 데이터 전처리에 정규표현식을 많이 사용한다.

In [2]:
# naddray 객체 생성
import numpy as np
dense = np.array([[3,0,1], [0,2,0]])
dense

array([[3, 0, 1],
       [0, 2, 0]])

In [3]:
# 희소행렬 - COO 형식
from scipy import sparse
data = np.array([3,1,2]) # 0이 아닌 데이터
row_pos = np.array([0,0,1]) #0이 아닌 데이터의 row 좌표
col_pos = np.array([0,2,1]) #0이 아닌 데이터의 col 좌표
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos))) 
# coo_matrix에 0이 아닌 데이터와 좌표를 인자로 넣어준다. coo형식으로 반환해준다.
print(sparse_coo)

  (0, 0)	3
  (0, 2)	1
  (1, 1)	2


In [4]:
# 희소행렬 - COO vs CSR 형식
dense2 = np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]])
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])
row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])
# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2,(row_pos,col_pos)))
print(sparse_coo)
print(sparse_coo.toarray())
print()
# CSR 형식으로 변환
# 행 위치 배열의 고유한 값들의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13])
sparse_csr = sparse.csr_matrix((data2,col_pos,row_pos_ind))
print(sparse_csr)
print(sparse_csr.toarray())

  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]

  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [13]:
# 희소행렬 - CSR 형식
dense2 = np.array([[0,0,1,0,0,5], [1,4,0,3,2,5], [0,6,0,3,0,0], \
                   [2,0,0,0,0,0], [0,0,0,7,0,8], [1,0,0,0,0,0]])

data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

In [14]:
dense2.shape

(6, 6)

In [6]:
# COO 형식으로 변환
row_pos = np.array([1,1,2,2,2,2,2,3,3,4,5,5,6])
col_pos = np.array([3,6,1,2,4,5,6,2,4,1,4,6,1])
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))
print(sparse_coo)
print(sparse_coo.toarray())

  (1, 3)	1
  (1, 6)	5
  (2, 1)	1
  (2, 2)	4
  (2, 4)	3
  (2, 5)	2
  (2, 6)	5
  (3, 2)	6
  (3, 4)	3
  (4, 1)	2
  (5, 4)	7
  (5, 6)	8
  (6, 1)	1
[[0 0 0 0 0 0 0]
 [0 0 0 1 0 0 5]
 [0 1 4 0 3 2 5]
 [0 0 6 0 3 0 0]
 [0 2 0 0 0 0 0]
 [0 0 0 0 7 0 8]
 [0 1 0 0 0 0 0]]


In [7]:
# CSR 형식으로 변환
# 행 위치 배열(row_pos)의 고유한 값들의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13]) #13은 전체 갯수를 의미
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))
print(sparse_csr)
print(sparse_csr.toarray())

  (0, 3)	1
  (0, 6)	5
  (1, 1)	1
  (1, 2)	4
  (1, 4)	3
  (1, 5)	2
  (1, 6)	5
  (2, 2)	6
  (2, 4)	3
  (3, 1)	2
  (4, 4)	7
  (4, 6)	8
  (5, 1)	1
[[0 0 0 1 0 0 5]
 [0 1 4 0 3 2 5]
 [0 0 6 0 3 0 0]
 [0 2 0 0 0 0 0]
 [0 0 0 0 7 0 8]
 [0 1 0 0 0 0 0]]


In [8]:
# DicVectorizer : 문서에서 단어의 사용빈도를 나타내는 딕셔너리 정보를 입력받아 BOW 인코딩한 수치 벡터로 전환
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
D = [{'A':1, 'B':2}, {'B':3, 'C':1}]
X = v.fit_transform(D)
print(X)
print(v.feature_names_) #피처의 이름
print(v.vocabulary_) # 피처의 순서
#사전에 D가 없기때문에 D가 출력이 안된다
# D가 나오게 하려면 다시 D를 집어넣고 fit_transform을 해줘야 한다.
print(v.transform({'C':4, 'D':3})) 

[[1. 2. 0.]
 [0. 3. 1.]]
['A', 'B', 'C']
{'A': 0, 'B': 1, 'C': 2}
[[0. 0. 4.]]


In [9]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This is the second document',
    'Ant the third one.',
    'Is this the first document?',
    'The last document?'
]

In [10]:
vect = CountVectorizer()
vect.fit(corpus)
print(vect.get_feature_names())
print()
# 알파벳 순으로 숫자 매겨줌
print(vect.vocabulary_)

['ant', 'document', 'first', 'is', 'last', 'one', 'second', 'the', 'third', 'this']

{'this': 9, 'is': 3, 'the': 7, 'first': 2, 'document': 1, 'second': 6, 'ant': 0, 'third': 8, 'one': 5, 'last': 4}


In [11]:
print(vect.transform(['This is the second document']).toarray())

[[0 1 0 1 0 0 1 1 0 1]]


In [12]:
print(vect.transform(['Something completely new.']).toarray())

[[0 0 0 0 0 0 0 0 0 0]]


In [13]:
print(vect.transform(corpus).toarray())

[[0 1 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 1 1 0 1]
 [1 0 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 1 0 0]]


In [14]:
# Stop Words는 문서에서 단어장을 생성할 때 무시할 수 있는 단어.
# 보통 영어의 관사, 접속사 한국어의 조사 등

vect = CountVectorizer(stop_words=['and', 'is', 'the', 'this']).fit(corpus)
vect.vocabulary_

{'first': 2,
 'document': 1,
 'second': 5,
 'ant': 0,
 'third': 6,
 'one': 4,
 'last': 3}

In [15]:
vect = CountVectorizer(stop_words='english').fit(corpus)
vect.vocabulary_

{'document': 1, 'second': 2, 'ant': 0}

In [16]:
# analyzer, tokenizer, token_pattern 등의 인수로 사용할 토큰 생성기를 선택
vect = CountVectorizer(analyzer = "char").fit(corpus)
vect.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [17]:
vect = CountVectorizer(token_pattern='t\w+').fit(corpus)
vect.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [18]:
# n-그램은 단어장 생성에 사용할 토큰의 크기 결정
vect = CountVectorizer(ngram_range=(1,2), token_pattern='t\W+').fit(corpus)
vect.vocabulary_

{'t ': 0, 't.': 3, 't  t.': 1, 't?': 4, 't  t?': 2}

In [19]:
vect = CountVectorizer(ngram_range=(1,2),token_pattern='t\w+').fit(corpus)
vect.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

TF-IDF(Term Frequency – Inverse Document Frequency) 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소
문서 d(document)와 단어 t 에 대해 다음과 같이 계산

tf-idf(d,t)=tf(d,t)⋅idf(t)

tf(d,t): term frequency. 특정한 단어의 빈도수
idf(t) : inverse document frequency. 특정한 단어가 들어 있는 문서의 수에 반비례하는 수
n : 전체 문서의 수
df(t) : 단어 t 를 가진 문서의 수
idf(d,t)=log(n/(1+df(t))

In [ ]:
corpus = ["""
The Corpus of Contemporary American English (COCA) is the only large, 
genre-balanced corpus of American English. COCA is probably the most 
widely-used corpus of English, and it is related to many other corpora of 
English that we have created, which offer unparalleled insight into variation 
in English.
"""]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidv = TfidfVectorizer(stop_words='english').fit(corpus)
print(tfidv.get_feature_names())
print()
print(tfidv.vocabulary_)
print()
print(tfidv.transform(corpus).toarray())
print(tfidv.transform(['This is the second document']).toarray())

['ant', 'document', 'second']

{'document': 1, 'second': 2, 'ant': 0}

[[0.         1.         0.        ]
 [0.         0.49084524 0.87124678]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]]
[[0.         0.49084524 0.87124678]]
